In [1]:
%load_ext autoreload
%autoreload 1

import os
from pathlib import Path
import glob
import random
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader
import torchvision.transforms as T

%aimport DataScienceBowl
from DataScienceBowl import DataScienceBowl
%aimport Trainer
from Trainer import Trainer
%aimport CustomUnet
from CustomUnet import CustomUnet

In [2]:
# Set device to GPU if available
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('using cuda')
else:
    torch.device('cpu')
    print('using cpu')
    
# Set seeds
seed = 42
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

using cuda


In [3]:
model = CustomUnet()
model.to(device)

CustomUnet(
  (model): Unet(
    (encoder): ResNetEncoder(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): Batc

In [4]:
transform = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
])

In [5]:
# Load data and split into train and test
dataset = DataScienceBowl('data/data_science_train', transform=transform)

indices = torch.randperm(len(dataset)).tolist()
N = len(indices)
train_test, test_unlab = map(int, (0.3 * N, 0.4 * N))

train_dataset = torch.utils.data.Subset(dataset, indices[:train_test])
test_dataset = torch.utils.data.Subset(dataset, indices[train_test:test_unlab])
unlabeled_dataset = torch.utils.data.Subset(dataset, indices[test_unlab:])

print(len(dataset))
print(len(train_dataset))
print(len(test_dataset))
print(len(unlabeled_dataset))

dataLoader_training = DataLoader(dataset=train_dataset, batch_size=8, shuffle=True)
dataLoader_test = DataLoader(dataset=test_dataset, batch_size=1, shuffle=True)
dataLoader_unlabeled = DataLoader(dataset=unlabeled_dataset, batch_size=8, shuffle=True)


670
201
67
402


In [8]:
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

trainer = Trainer(model=model,
                 device=device,
                 criterion=criterion,
                 optimizer=optimizer,
                 training_DataLoader=dataLoader_training,
                 test_DataLoader=dataLoader_test,
                 epochs=20,
                 notebook=True)

train_loss, _, lr, test_loss, test_iou = trainer.run_trainer()


Progress:   0%|          | 0/20 [00:00<?, ?it/s]

Training:   0%|          | 0/26 [00:00<?, ?it/s]

Training:   0%|          | 0/26 [00:00<?, ?it/s]

Training:   0%|          | 0/26 [00:00<?, ?it/s]

Training:   0%|          | 0/26 [00:00<?, ?it/s]

...

Training:   0%|          | 0/26 [00:00<?, ?it/s]

Training:   0%|          | 0/26 [00:00<?, ?it/s]

Training:   0%|          | 0/26 [00:00<?, ?it/s]

Training:   0%|          | 0/26 [00:00<?, ?it/s]

Test:   0%|          | 0/67 [00:00<?, ?it/s]

In [9]:
print(test_iou)

0.7035618578632846
